# Task for DA Intern Position at SberMarket

## 1. SQL

**Objective**: Identify the department head whose employee currently receives the highest salary.

**Data Structure**:

Table `employees`:
- `head` - department head (each head is unique)
- `employee` - employee (each employee is unique)

Table `salaries`:
- `employee` - employee (each employee is unique)
- `salary` - salary
- `date` - salary update date

**Sample Data**:

The following data is provided for testing purposes:

```sql
CREATE TABLE salaries(
    employee VARCHAR(5),
    salary INT,
    date DATE
);

INSERT INTO salaries(employee, salary, date) VALUES ('tom', '2342', '2023-01-14');
INSERT INTO salaries(employee, salary, date) VALUES ('tom', '2345', '2023-03-17');
INSERT INTO salaries(employee, salary, date) VALUES ('nick', '5424', '2022-04-01');
INSERT INTO salaries(employee, salary, date) VALUES ('nick', '5500', '2023-03-01');
INSERT INTO salaries(employee, salary, date) VALUES ('juli', '4566', '2021-04-03');
INSERT INTO salaries(employee, salary, date) VALUES ('juli', '6000', '2024-01-12');
INSERT INTO salaries(employee, salary, date) VALUES ('axel', '12444', '2022-05-17');
INSERT INTO salaries(employee, salary, date) VALUES ('axel', '14000', '2023-11-12');
INSERT INTO salaries(employee, salary, date) VALUES ('henry', '1244', '2021-03-19');
INSERT INTO salaries(employee, salary, date) VALUES ('henry', '7000', '2023-02-09');
INSERT INTO salaries(employee, salary, date) VALUES ('tomas', '2345', '2024-02-02');
INSERT INTO salaries(employee, salary, date) VALUES ('tomas', '4000', '2024-02-09');
INSERT INTO salaries(employee, salary, date) VALUES ('fiona', '1000', '2023-07-31');
INSERT INTO salaries(employee, salary, date) VALUES ('fiona', '1200', '2024-01-29');
INSERT INTO salaries(employee, salary, date) VALUES ('ben', '1244', '2021-12-14');
INSERT INTO salaries(employee, salary, date) VALUES ('ben', '7001', '2023-03-27');
INSERT INTO salaries(employee, salary, date) VALUES ('shen', '6000', '2022-02-18');
INSERT INTO salaries(employee, salary, date) VALUES ('shen', '7001', '2024-01-28');
INSERT INTO salaries(employee, salary, date) VALUES ('lora', '4050', '2023-08-30');
INSERT INTO salaries(employee, salary, date) VALUES ('lora', '7400', '2024-01-19');

CREATE TABLE employees(
    head VARCHAR(6),
    employee VARCHAR(5)
);

INSERT INTO employees(head, employee) VALUES ('donald', 'tom');
INSERT INTO employees(head, employee) VALUES ('olaf', 'nick');
INSERT INTO employees(head, employee) VALUES ('jacob', 'juli');
INSERT INTO employees(head, employee) VALUES ('donald', 'axel');
INSERT INTO employees(head, employee) VALUES ('olaf', 'henry');
INSERT INTO employees(head, employee) VALUES ('jacob', 'tomas');
INSERT INTO employees(head, employee) VALUES ('donald', 'fiona');
INSERT INTO employees(head, employee) VALUES ('olaf', 'ben');
INSERT INTO employees(head, employee) VALUES ('jacob', 'shen');
INSERT INTO employees(head, employee) VALUES ('doanld', 'lora');
```

### Solution

```sql
-- Find the most recent (by date) salary for each employee
WITH LatestSalaries AS (
    SELECT 
        employee,
        salary
    FROM 
        salaries s1
    WHERE 
        date = (SELECT MAX(date) 
                FROM salaries s2 
                WHERE s1.employee = s2.employee)
)


-- Join the most recent salaries with the employees table, sort by salary, and output the department head's name
SELECT 
    e.head
FROM 
    LatestSalaries ls
JOIN 
    employees e ON ls.employee = e.employee
ORDER BY 
    ls.salary DESC
LIMIT 1;
```

## 2. Pandas

**Objective**: Imagine that the database management system is not working, but you have two files `employees.csv` and `salaries.csv`. Find the department head whose employee currently receives the highest salary.

In [1]:
import pandas as pd

employees = [['donald', 'tom'],
             ['olaf', 'nick'],
             ['jacob', 'juli'],
             ['donald', 'axel'],
             ['olaf', 'henry'],
             ['jacob', 'tomas'],
             ['donald', 'fiona'],
             ['olaf', 'ben'],
             ['jacob', 'shen'],
             ['doanld', 'lora']]

employees = pd.DataFrame(employees, columns=['head', 'employee'])

salaries = [['tom', 2342, '2023-01-14'],
            ['tom', 2345, '2023-03-17'],
            ['nick', 5424, '2022-04-01'],
            ['nick', 5500, '2023-03-01'],
            ['juli', 4566, '2021-04-03'],
            ['juli', 6000, '2024-01-12'],
            ['axel', 12444, '2022-05-17'],
            ['axel', 14000, '2023-11-12'],
            ['henry', 1244, '2021-03-19'],
            ['henry', 7000, '2023-02-09'],
            ['tomas', 2345, '2024-02-02'],
            ['tomas', 4000, '2024-02-09'],
            ['fiona', 1000, '2023-07-31'],
            ['fiona', 1200, '2024-01-29'],
            ['ben', 1244, '2021-12-14'],
            ['ben', 7001, '2023-03-27'],
            ['shen', 6000, '2022-02-18'],
            ['shen', 7001, '2024-01-28'],
            ['lora', 4050, '2023-08-30'],
            ['lora', 7400, '2024-01-19']]

salaries = pd.DataFrame(salaries, columns=['employee', 'salary', 'date'])

### Solution

In [2]:
# Transform date to datetime
salaries['date'] = pd.to_datetime(salaries['date'])

# Sort by date and keep only the latest salary for each employee
latest_salaries = salaries.sort_values(by='date', ascending=False).drop_duplicates('employee', keep='first')

# Merge employees and salaries
merged = latest_salaries.merge(employees, on='employee')

# Find the department head whose employee currently has the highest salary
result = merged.loc[merged['salary'].idxmax()]
print(f'The department head whose employee currently receives the highest salary is {result["head"].capitalize()}')

The department head whose employee currently receives the highest salary is Donald


## 3. Parsing

**Objective**: Write a function to parse the <a href="https://www.avito.ru/">Avito</a> website that takes a car brand and model as input and returns a DataFrame with the relevant offers. The DataFrame should include the offer title, the link to the offer page, and the price. You are allowed to use any libraries for parsing.

### Solution

In [3]:
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [4]:
def avito_parser(brand: str, model: str) -> pd.DataFrame:
    """
    Function to parse car offers from avito.ru for a specific brand and model.

    Parameters:
    brand (str): The car brand to search for.
    model (str): The car model to search for.

    Returns:
    pd.DataFrame: DataFrame containing the offer title, link to the offer, and the car price.
    """

    # Set up logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)
    logger.info(f'Starting parser for brand: {brand}, model: {model}.')

    # Configure the browser in headless mode
    options = Options()
    options.add_argument('--headless=new')
    driver = webdriver.Chrome(options=options)

    try:
        # Form the URL and navigate to the offers page
        url = f'https://www.avito.ru/all/avtomobili/{brand}/{model}'
        logger.info(f'Navigating to page: {url}.')
        driver.get(url)
        driver.implicitly_wait(10)

        # Find all offers on the page
        offers = driver.find_elements(By.XPATH, '//div[@data-marker="item"]')
        logger.info(f'Found {len(offers)} offers.')

        # Create an empty DataFrame to store the information
        df = pd.DataFrame(columns=['Title', 'Link', 'Price, RUB'])

        # Set maximum display length for strings
        pd.set_option('display.max_colwidth', None)

        # Gather information about each offer
        for i, offer in enumerate(offers):
            try:
                # Find the title, link, and price
                title = offer.find_element(By.XPATH, './/h3[@itemprop="name"]').text
                link = offer.find_element(By.XPATH, './/a[@itemprop="url"]').get_attribute('href')
                price = offer.find_element(By.XPATH, './/meta[@itemprop="price"]').get_attribute('content')
                price_formatted = '{:,}'.format(int(price)).replace(',', ' ')

                # Add the information to the DataFrame
                df.loc[len(df)] = {'Title': title, 'Link': link, 'Price, RUB': price_formatted}
                logger.info(f'Processed offer {i+1}: {title}, {price_formatted} RUB.')
            except Exception as e:
                logger.error(f'Error processing offer {i+1}: {e}.')

    except Exception as e:
        logger.error(f'Error accessing Avito: {e}.')

    finally:
        # Close the browser
        driver.quit()
        logger.info('Parsing completed.')

    return df

In [5]:
# Test the function
brand = 'mercedes-benz'
model = 's-klass_amg'
df = avito_parser(brand, model)

2024-05-23 16:05:06,784 - INFO - Starting parser for brand: mercedes-benz, model: s-klass_amg.
2024-05-23 16:05:13,723 - INFO - Navigating to page: https://www.avito.ru/all/avtomobili/mercedes-benz/s-klass_amg.
2024-05-23 16:05:27,129 - INFO - Found 53 offers.
2024-05-23 16:05:28,250 - INFO - Processed offer 1: Mercedes-Benz S-класс AMG 5.5 AT, 2013, 67 757 км, 5 259 000 RUB.
2024-05-23 16:05:28,666 - INFO - Processed offer 2: Mercedes-Benz S-класс AMG 5.5 AT, 2015, 165 223 км, 5 600 000 RUB.
2024-05-23 16:05:29,073 - INFO - Processed offer 3: Mercedes-Benz S-класс AMG 4.0 AT, 2023, 4 227 км, 31 300 000 RUB.
2024-05-23 16:05:34,450 - INFO - Processed offer 4: Mercedes-Benz S-класс AMG 4.0 AT, 2023, 20 км, 26 500 000 RUB.
2024-05-23 16:05:34,914 - INFO - Processed offer 5: Mercedes-Benz S-класс AMG 5.5 AT, 2015, 115 000 км, 6 899 999 RUB.
2024-05-23 16:05:37,722 - INFO - Processed offer 6: Mercedes-Benz S-класс AMG 5.5 AT, 2015, 137 000 км, 5 249 000 RUB.
2024-05-23 16:05:38,079 - INFO 

In [6]:
# Check the results
df.head()

,Title,Link,"Price, RUB"
0,"Mercedes-Benz S-класс AMG 5.5 AT, 2013, 67 757 км",https://www.avito.ru/moskva/avtomobili/mercedes-benz_s-klass_amg_5.5_at_2013_67_757_km_3653843460,5 259 000
1,"Mercedes-Benz S-класс AMG 5.5 AT, 2015, 165 223 км",https://www.avito.ru/sankt-peterburg/avtomobili/mercedes-benz_s-klass_amg_5.5_at_2015_165_223_km_3784600424,5 600 000
2,"Mercedes-Benz S-класс AMG 4.0 AT, 2023, 4 227 км",https://www.avito.ru/moskva/avtomobili/mercedes-benz_s-klass_amg_4.0_at_2023_4_227_km_3920570627,31 300 000
3,"Mercedes-Benz S-класс AMG 4.0 AT, 2023, 20 км",https://www.avito.ru/moskva/avtomobili/mercedes-benz_s-klass_amg_4.0_at_2023_20_km_3850831415,26 500 000
4,"Mercedes-Benz S-класс AMG 5.5 AT, 2015, 115 000 км",https://www.avito.ru/moskva/avtomobili/mercedes-benz_s-klass_amg_5.5_at_2015_115_000_km_3848979273,6 899 999


In [7]:
# Save the results to a CSV file
output_file = f'Avito_{brand}_{model}.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')